# Exploring Regulatory Sequence of *tetR*/*tetA* in Tn10 

(c) 2020 Tom Röschinger. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

***

In this notebook we look at the transposon **tn10**, which contains a natural system for the expression of *tetA*, which is regulated by *tetR*.

In [1]:
import wgregseq

# Include these if package is manipulated while running the notebook
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

First we read the FASTA file obtained from Genebank.

In [2]:
with open ("tn10.fasta", "r") as file:
    data = file.read().split('\n')[1:]
    sequence = "".join(data)

Organization of tetR/tetA regulation:
- two operators that can be bound independently by TetR
- tetA is repressed by both tetO1 and tetO2
- tetR is repressed only by tetO1
- Affinity of tetO2 to TetR is about twice as high as tetO1

![](tn10_tet.png)

From Genebank, we can find the positions for *tetA* and *tetR*. The repressor gene is reversed, so we will have to obtain the complementary sequence in case we are interested in the actual sequences.

In [3]:
# Exact positions from Genebank
tetR_pos = [4702, 5328]
tetA_pos = [5407, 6612]

For simple access, let's extract the region between the two genes, which contains all regulatory elements.

In [4]:
intergenic_region = sequence[5328:5407-1]
intergenic_region_rev = wgregseq.complement_seq(intergenic_region, rev=True)
intergenic_region

'TAATTCCTAATTTTTGTTGACACTCTATCATTGATAGAGTTATTTTACCACTCCCTATCAGTGATAGAGAAAAGTGAA'

In [5]:
len(intergenic_region)

78

Let's extract the sequences for the operators and promoters to confirm we found the right indices.

In [6]:
tetO1 = intergenic_region[21:40]
print("tetO1: ", tetO1)

tetO2 = intergenic_region[51:70]
print("tetO2: ", tetO2)

tetO1:  ACTCTATCATTGATAGAGT
tetO2:  TCCCTATCAGTGATAGAGA


In [7]:
rev_tetO1 = wgregseq.complement_seq(tetO1, rev=True)
rev_tetO1

'ACTCTATCAATGATAGAGT'

In [8]:
rev_tetO2 = wgregseq.complement_seq(tetO2, rev=True)
rev_tetO2

'TCTCTATCACTGATAGGGA'

In [9]:
P_tetA = intergenic_region[16:53]
P_tetA

'TTGACACTCTATCATTGATAGAGTTATTTTACCACTC'

In [10]:
P_tetR1 = intergenic_region_rev[7:45]
P_tetR1

'TTCTCTATCACTGATAGGGAGTGGTAAAATAACTCTAT'

In [11]:
P_tetR2 = intergenic_region_rev[28:65]
P_tetR2

'TGGTAAAATAACTCTATCAATGATAGAGTGTCAACAA'

These all look fine. Let's assign the lacUV5 sequence from Brewster 2012 to a variable, so we can add it to the mutated sequences later on.

In [12]:
lacUV5 = 'TCGAGTTTACACTTTATGCTTCCGGCTCGTATAATGTGTGG'

## Constructs

All constructs which are include a tet operator need to be integrated into a cell which expresses the tet repressor. However, the inserts which only have the promoter should be observed in the absence of the repressor to identify the binding energy matrix for the -10/-35 regions.

### LacUV5 + individual operators downstream

First we mutate the operator sequences and put them downstream of lacUV5, such that we obtain a simple repression motif. Therefore, we use the function `wgregseq.mutations_det` to generate single and double mutants of each operator. We include all single mutants and 400 single mutants. The function guarantees that only unique sequences are returned, so no duplicates.

First the tetO1 single mutants.

In [13]:
# Obtain mutants
mutants_single = wgregseq.mutations_det(tetO1, mut_per_seq=1)

# Store sequences in data frame
tetO1_df_single = pd.DataFrame({"seq":mutants_single})

# Add description column
tetO1_df_single["construct"] = "lacUV5_tetO1 single mutant"

# Show first 5 rows
tetO1_df_single.head()

,seq,construct
0,cCTCTATCATTGATAGAGT,lacUV5_tetO1 single mutant
1,AaTCTATCATTGATAGAGT,lacUV5_tetO1 single mutant
2,ACaCTATCATTGATAGAGT,lacUV5_tetO1 single mutant
3,ACTaTATCATTGATAGAGT,lacUV5_tetO1 single mutant
4,ACTCaATCATTGATAGAGT,lacUV5_tetO1 single mutant


Second the tetO1 double mutants.

In [14]:
# Obtain mutants
mutants_double_O1 = wgregseq.mutations_det(tetO1, mut_per_seq=2, num_mutants=400, site_start=-20)

# Store sequences in data frame
tetO1_df_double = pd.DataFrame({"seq":mutants_double_O1})

# Add description column
tetO1_df_double["construct"] = "lacUV5_tetO1 double mutant"

# Show first 5 rows
tetO1_df_double.head()

,seq,construct
0,ACTCTAgCATTtATAGAGT,lacUV5_tetO1 double mutant
1,ACTCTATCgTTGATcGAGT,lacUV5_tetO1 double mutant
2,ACTCaATCAgTGATAGAGT,lacUV5_tetO1 double mutant
3,cCTCaATCATTGATAGAGT,lacUV5_tetO1 double mutant
4,ACTaTATCAaTGATAGAGT,lacUV5_tetO1 double mutant


Since we are downsampling the double mutants, we should make sure that we obtain sufficient mutational coverage across the sequence. Hence, we use `wgregseq.mutation_coverage` to compute the mutation rate at each position. If there are positions which significantly vary of 0.1, we either have to rerun the function which generates the mutations, or take a larger sample size.

In [15]:
wgregseq.mutation_coverage(tetO1, mutants_double_O1)

array([0.1275, 0.0975, 0.115 , 0.0975, 0.09  , 0.1075, 0.1075, 0.1   ,
       0.1025, 0.115 , 0.11  , 0.0775, 0.12  , 0.1075, 0.1075, 0.1075,
       0.125 , 0.0875, 0.0975])

Now we can generate the mutants for tetO2 as well.

In [16]:
# Single mutants
mutants_single_O2 = wgregseq.mutations_det(tetO2, mut_per_seq=1)
tetO2_df_single = pd.DataFrame({"seq":mutants_single})
tetO2_df_single["construct"] = "lacUV5_tetO2 single mutant"

# Double mutants
mutants_double_O2 = wgregseq.mutations_det(tetO2, mut_per_seq=2, num_mutants=400)
tetO2_df_double = pd.DataFrame({"seq":mutants_double_O2})
tetO2_df_double["construct"] = "lacUV5_tetO2 double mutant"

Again, before we proceed we should check the mutation coverage.

In [17]:
wgregseq.mutation_coverage(tetO2, mutants_double_O2)

array([0.11  , 0.095 , 0.1325, 0.1175, 0.1125, 0.085 , 0.0925, 0.0925,
       0.1175, 0.085 , 0.085 , 0.1025, 0.115 , 0.115 , 0.11  , 0.1175,
       0.1175, 0.095 , 0.1025])

Finally, we combine all sequences into a single data frame, and attach the lacUV5 promoter directly upstream of the mutated operators.

In [18]:
# Combine data frames
tet_df = pd.concat([tetO1_df_single, tetO1_df_double, tetO2_df_single, tetO2_df_double], ignore_index=True)

# Attach lacUV5 to each sequence
tet_df.seq = [lacUV5 + seq for seq in tet_df.seq]

# Print first 5 rows
tet_df.head()

,seq,construct
0,TCGAGTTTACACTTTATGCTTCCGGCTCGTATAATGTGTGGcCTCT...,lacUV5_tetO1 single mutant
1,TCGAGTTTACACTTTATGCTTCCGGCTCGTATAATGTGTGGAaTCT...,lacUV5_tetO1 single mutant
2,TCGAGTTTACACTTTATGCTTCCGGCTCGTATAATGTGTGGACaCT...,lacUV5_tetO1 single mutant
3,TCGAGTTTACACTTTATGCTTCCGGCTCGTATAATGTGTGGACTaT...,lacUV5_tetO1 single mutant
4,TCGAGTTTACACTTTATGCTTCCGGCTCGTATAATGTGTGGACTCa...,lacUV5_tetO1 single mutant


This data frame will be included in the final file for the twist order. Therefore, we need to include a column which contains the information if we already added primers to the sequences. Then we save the table as `csv` file in the appropriate folder in this repository.

In [19]:
# Add column
tet_df['primer_added'] = False

# Store file
tet_df.to_csv("../../../../data/twist_order/lacUV5_tetOx_single_double_mutants.csv")

## Native Promoter sequences

Next, we design the constructs which contain the native promoter sequences. The goal here is to obtain an energy matrix for the RNAP binding sites. We do not need to be bothered about the operator binding sites, since we can integrate these constructs into strains which do not express the tet repressor and therefore the operators are not bound. In the paper there are two promoters annotated, but since we want to obtain data for the promoters individually, we will randomize the -10/-35 region of the opposing promoter if possible.

First we consider the promoter P_tetR1. Since the -35 region of P_tetR2 overlaps with the -10 of P_tetR2, we can only randomize the -10. Note that we are looking at the reverse sequence, since *tetR* is transcribed in the reverse direction.

In [20]:
# Obtain reversed intergenic region as list to manipulate positions
randomized_R2 = list(intergenic_region_rev)

# Randomize -10 of P_tetR2
randomized_R2[50:66] = list(wgregseq.gen_rand_seq(16).lower())

# Recombine into string
randomized_R2 = "".join(randomized_R2)

# Print result
randomized_R2

'TTCACTTTTCTCTATCACTGATAGGGAGTGGTAAAATAACTCTATCAATGcaagttccgtaatcctAATTAGGAATTA'

Now we can generate mutations in P_tetR1. We'll randomly mutate the RNAP binding site at an average 0.1 rate (the actual number of mutants is picked from a Poisson distribution with mean `len(sequence) * rate`. The number of mutations can also be fixed, refer to the docstring of the function.) After generating mutations, we check the mutation coverage.

In [21]:
# Generate mutants
PR1_mutants = np.unique(wgregseq.mutations_rand(randomized_R2, 1000, 0.1, site_start=6, site_end=46))

# Check mutation coverage
wgregseq.mutation_coverage(randomized_R2, PR1_mutants, site_start=6, site_end=46)

array([0.12344398, 0.10892116, 0.0840249 , 0.10788382, 0.10580913,
       0.10788382, 0.10995851, 0.09958506, 0.09024896, 0.1120332 ,
       0.08817427, 0.093361  , 0.09958506, 0.11410788, 0.11099585,
       0.10580913, 0.12240664, 0.10373444, 0.11929461, 0.08091286,
       0.08609959, 0.09958506, 0.10373444, 0.12551867, 0.13381743,
       0.10165975, 0.11825726, 0.09439834, 0.093361  , 0.08609959,
       0.10995851, 0.1026971 , 0.07987552, 0.09543568, 0.09024896,
       0.09958506, 0.09439834, 0.09751037, 0.10477178, 0.08506224])

Next, we mutate the P_tetR2 region. Therefore, we first have to randomize the -35 region of P_tetR1. 

In [22]:
# Obtain reversed intergentic region as list
randomized_R1 = list(intergenic_region_rev)

# Randomize P_tetR1 -35 region
randomized_R1[6:21] = list(wgregseq.gen_rand_seq(16).lower())

# Recombine into string
randomized_R1 = "".join(randomized_R1)

# Print sequence
randomized_R1

'TTCACTcgttcagcgcgctcagTAGGGAGTGGTAAAATAACTCTATCAATGATAGAGTGTCAACAAAAATTAGGAATTA'

Again, we generate mutants of the RNAP binding site and compute the mutation rate per position as quality control.

In [23]:
# Generate mutants
PR2_mutants = np.unique(wgregseq.mutations_rand(randomized_R1, 1000, 0.1, site_start=27, site_end=66))

# Compute mutation rate per position
wgregseq.mutation_coverage(randomized_R1, PR2_mutants, site_start=27, site_end=66)

array([0.10878661, 0.10669456, 0.10983264, 0.10774059, 0.10146444,
       0.12343096, 0.11820084, 0.08786611, 0.10774059, 0.09728033,
       0.09518828, 0.08472803, 0.10041841, 0.08786611, 0.10460251,
       0.11610879, 0.12133891, 0.10564854, 0.09937238, 0.11087866,
       0.11820084, 0.11506276, 0.09728033, 0.10460251, 0.11610879,
       0.10355649, 0.09937238, 0.10774059, 0.11297071, 0.11506276,
       0.10564854, 0.09623431, 0.10041841, 0.10041841, 0.09414226,
       0.10251046, 0.10146444, 0.08891213, 0.10355649])

Finally, we get mutants for the P_tetA sequence. Since this is the only RNAP binding site in the forward direction, we do not need to randomize anything of the sequence prior to generating mutants.

In [24]:
# Generate mutants
PA_mutants = np.unique(wgregseq.mutations_rand(intergenic_region, 1000, 0.1, site_start=16, site_end=53))

# Compute mutation rate per position
wgregseq.mutation_coverage(intergenic_region, PA_mutants, site_start=16, site_end=53)

array([0.09168443, 0.1130064 , 0.09594883, 0.10021322, 0.1119403 ,
       0.1130064 , 0.1098081 , 0.10021322, 0.10660981, 0.10554371,
       0.1119403 , 0.10767591, 0.10660981, 0.11407249, 0.11833689,
       0.1098081 , 0.10767591, 0.11620469, 0.10234542, 0.12046908,
       0.1098081 , 0.07995736, 0.10447761, 0.1130064 , 0.09381663,
       0.09808102, 0.11513859, 0.09914712, 0.09701493, 0.09061834,
       0.09808102, 0.09808102, 0.09808102, 0.10447761, 0.12260128,
       0.10447761, 0.10127932])

We combine all generated sequences into a data frame and add the necessary columns. Also we add a column that indicates if there is a region in the sequence that was randomized on top of the mutations in the sequence of interest.

In [25]:
# Define individual data frames
dfR1 = pd.DataFrame({'seq': PR1_mutants, 'primer_added' : False, 'construct' : "P_tetR1", 'note': "P_tetR2 -10 region randomized"})
dfR2 = pd.DataFrame({'seq': PR2_mutants, 'primer_added' : False, 'construct' : "P_tetR2", 'note': "P_tetR1 -35 region randomized"})
dfA = pd.DataFrame({'seq': PA_mutants, 'primer_added' : False, 'construct' : "P_tetA", 'note': ""})

# Combine data frames
df_promoters = pd.concat([dfR1, dfR2, dfA], ignore_index=True)
df_promoters

,seq,primer_added,construct,note
0,TTCACTTTTCTCTATCACTGATAGGGAGTGGTAAAATAACTCTATC...,False,P_tetR1,P_tetR2 -10 region randomized
1,TTCACTTTTCTCTATCACTGATAGGGAGTGGTAAAATAACTCTAgC...,False,P_tetR1,P_tetR2 -10 region randomized
2,TTCACTTTTCTCTATCACTGATAGGGAGTGGTAAAATAACTCTcTC...,False,P_tetR1,P_tetR2 -10 region randomized
3,TTCACTTTTCTCTATCACTGATAGGGAGTGGTAAAATAACTCaATC...,False,P_tetR1,P_tetR2 -10 region randomized
4,TTCACTTTTCTCTATCACTGATAGGGAGTGGTAAAATAACTgTATC...,False,P_tetR1,P_tetR2 -10 region randomized
...,...,...,...,...
2853,TAATTCCTAATTTTTGgaGACACTCTATCATTGATgGAaTTATTgT...,False,P_tetA,
2854,TAATTCCTAATTTTTGgaGACACTCTAaCATTGATAtAaTTATTTT...,False,P_tetA,
2855,TAATTCCTAATTTTTGgaGACACgCTATCATTGATAtAcTTgcTTT...,False,P_tetA,
2856,TAATTCCTAATTTTTGgaGACgCTCTATCAaTGATAGAGTgATTTT...,False,P_tetA,


The combined data frame is saved in the appropriate folder where we are combining the individual constructs into a final order.

In [26]:
tet_df.to_csv("../../../../data/twist_order/natural_tet_promoters_mutated.csv")

## Computational environment

In [27]:
%load_ext watermark
%watermark -v -p pandas,wgregseq

CPython 3.8.5
IPython 7.10.0

pandas 1.0.3
wgregseq 0.0.1
